In [2]:

# Import classes for getting serial data
import serial
import serial.tools.list_ports
import serial.tools.list_ports as port_list


# Import the DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

# Import classes for sending data via mqtt 
import paho.mqtt.client as mqtt
from time import sleep


In [13]:
#Function for training / calibration

def TrainData():

    #Import the dataset 

    # Important: add position data manually
    # Comma for floating point numbers

    
    datasetTrain_all = pd.read_csv("Train_2306_langsam.csv",sep=';')   #calibration / train data

    train_features1 =  datasetTrain_all.drop('Position',axis=1)   # drop column Position - used as target
    train_features =  train_features1.drop('Timer',axis=1)   # drop column Timer

    train_targets = datasetTrain_all["Position"] # use Position as target

    # Train the model

    tree = DecisionTreeClassifier(criterion = 'entropy').fit(train_features,train_targets)
    return tree   # return the resulting tree

In [4]:
# read the sensor information from the USB port and transform it to the needed array

def ConvertUSBtoArray(OneLineUSB):
    #initialization
    Array=np.empty((0),int)   
    SplitString=OneLineUSB.split(",")
    if (len(SplitString)>1):    #convert the data into the variables if data was received
                
                MagX=int(SplitString[1])
                MagY=int(SplitString[2])
                MagZ=int(SplitString[3])
                Light=int(SplitString[4])
                
                helpOneLine = [[MagX,  MagY, MagZ, Light]]
                
                ArrayOneLine = pd.DataFrame(helpOneLine, columns=['MagX','MagY','MagZ',"Light"])  #create the needed dataframe
    return(ArrayOneLine)

In [5]:
# initalize the serial port for receiving the sensor data

def getPort():

    ports = list(port_list.comports())
    for p in ports:   # get the port list - for information only
        print (p)


    ser = serial.Serial('COM3') #use this port - might differ from computer to computer

    return ser


In [6]:
#read out the USB data and make the prediction
def readUSB(ser):
    
    
    Bytes = ser.readline()
    OneLineUSB=Bytes.decode("utf-8")  # read the serial port
    print(OneLineUSB)  # for information only

    ArrayOneLine=ConvertUSBtoArray(OneLineUSB) # transform the data into the correct dataframe
    print(ArrayOneLine) # for information only

    prediction = tree.predict(ArrayOneLine) # make the prediction
    print(prediction) # for information only


    return(prediction)


In [9]:
def FilterPrediction(ser,tree):
    
    # initialize the values, also using some filters
    count=0
    mean=0
    mean_old=-1
    filter_i=-1
    glaettungswert = 0.1 # maximum change per step
    mean_count=10
    max_wert=4.3
    min_wert=0


       # The callback for when the client receives a CONNACK response from the server.
    def on_connect(client, userdata, flags, rc):
        print("Connected with result code "+str(rc))

        # Subscribing in on_connect() means that if we lose the connection and
        # reconnect then subscriptions will be renewed.
        client.subscribe("/Azure/Stand/Data")
        

    # The callback for when a PUBLISH message is received from the server.
    def on_message(client, userdata, msg):
        print(msg.topic+" "+str(msg.payload))


    def on_publish(client,userdata,result):             #create function for callback
        print("data published \n")
        pass

    # initialize the MQTT server
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message
    client.on_publish = on_publish  
    client.connect("xx.xx.xx.xxx", 1883,60) 

    # make the prediction for the next 5000 data points
    for i in range(1,5000):
        predictedValue=readUSB(ser) # get the information from the USB port
        print(predictedValue) # for information only
        predictedValue=predictedValue[0] #for filtering

        # mean-filter as well as reduce transmission rate
        if count%mean_count == 0 and count>0:   #for filtering
                mean=mean/mean_count
                if not(abs(mean_old-mean)<glaettungswert or mean_old==-1):  #limit the step width
                    if (mean_old-mean)<glaettungswert :
                        mean=mean_old+glaettungswert
                    else:
                        mean=mean_old-glaettungswert
        
                if mean < min_wert: 
                    mean=min_wert
    
                if mean > max_wert:
                    mean=max_wert
    
                # Calculate Difference an set mean_old
                if mean_old == -1: # at initialization
                    Difference=mean
                else:
                    Difference=abs(mean-mean_old)
              

                #Attention: "+" will be converted to "," in NodeRed and and decimal -"," as "."
                Data="{\"Position\":" + str(mean)+ "+ \"Difference\":" + str(Difference)+"}" 

                print(Data) # for information only
                ret = client.publish("/Azure/Stand/Data",Data)
                sleep(0.1) # for speed reduction
                
                #set mean_old and reset mean
                mean_old=mean               
                mean=0

        # filter pikes
        help_i=predictedValue.replace(",", ".")    
        help_i=float(help_i)
        if abs(filter_i-help_i)<glaettungswert or filter_i==-1:
            filter_i=help_i
        else:
            if (filter_i-help_i) < glaettungswert:
                filter_i=filter_i + glaettungswert
            else:
                filter_i=filter_i - glaettungswert          
        mean=mean+ filter_i
        count=count+1

In [15]:
#Stp 1: InitilizUSB-port
ser=getPort()





COM3 - XDK (COM3)


In [16]:
#Step 2: train the tree
tree=TrainData()

In [17]:
#Step 3: Make the prediction with the current USB information
FilterPrediction(ser,tree)


 

   MagX  MagY  MagZ  Light
0   -36   -37   -26  28800
['0,95']
['0,95']
8898, -35, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -35   -36   -24  28800
['0,95']
['0,95']
{"Position":0.95+ "Difference":0.0}
data published 

8929, -35, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -35   -36   -24  28800
['0,95']
['0,95']
8959, -35, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -35   -36   -24  28800
['0,95']
['0,95']
8990, -35, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -35   -36   -24  28800
['0,95']
['0,95']
9021, -36, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -36   -36   -24  28800
['0,95']
['0,95']
9051, -36, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -36   -36   -24  28800
['0,95']
['0,95']
9082, -36, -36, -24, 28800 

   MagX  MagY  MagZ  Light
0   -36   -36   -24  28800
['0,95']
['0,95']
9112, -36, -36, -26, 28800 

   MagX  MagY  MagZ  Light
0   -36   -36   -26  28800
['0,95']
['0,95']
9143, -36, -36, -26, 28800 

   MagX  MagY  MagZ  Light
0   -36 

KeyboardInterrupt: 

In [23]:
ser.close()